In [ ]:
# Block 0: Documentation

print('Script to process and visualize ABI aerosol optical depth (AOD) data\n')
print('Version 3.0, August 30, 2022\n')
print('Written using Python v3.9, netCDF4 v1.5.7, Matplotlib v3.5.1, and Cartopy v0.18.0.\n')
print('Author: Dr. Amy Huff (IMSG at NOAA/NESDIS/STAR), amy.huff@noaa.gov\n')
print('This script processes ABI AOD data and plots the data from a single file or a multi-file composite on a map to create a professional-looking figure. The user can choose to plot the full dataset on the native geostationary map projection, or a subset of the data on a zoomed-in Plate Carree map projection.\n')
print('Block 1 imports modules and libraries, and Blocks 2-16 are functions that require no input from the user; there is no visible output from these blocks. The user specifies processing/plotting settings in Block 17 and obtains output from Block 18.\n')
print('**Please acknowledge the NOAA/NESDIS/STAR Aerosols and Atmospheric Composition Science Team if using any of this code in your work/research!**')

In [ ]:
# Block 1: Import Python packages

# Library to perform array operations
import numpy as np

# Library to make plots
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker

# Library to create maps
from cartopy import crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.io.shapereader as shpreader

# Library to work with netCDF files
from netCDF4 import Dataset

# Module for manipulating dates and times
import datetime

# Module to set filesystem paths appropriate for user's operating system
from pathlib import Path

# Modules to create interactive menus in Jupyter Notebook
from IPython.display import display
import ipywidgets as widgets

# Library controlling warning messages
# Cartopy can trigger many irrelevant warning messages via Matplotlib
# Recommend ignorning these distracting warnings
import warnings
warnings.filterwarnings('ignore')

# Import supporting functions needed to run script
# These functions moved to external .py file to make training Notebook cleaner
import supporting_functions

In [ ]:
# Block 2: Process ABI AOD data from a single data file & calculate latitude/longitude from GOES ABI fixed grid projection data
# GOES ABI fixed grid projection is a map projection relative to the GOES satellite
# Units: latitude in °N (°S < 0), longitude in °E (°W < 0)
# See GOES-R Product User Guide (PUG) Volume 5 (L2 products) Section 4.2.8 for details & example of calculations
# "file_id": parameter variable assigned when the satellite data file is opened
# "quality": parameter variable from widget menus, set in main function

def abi_aod_data(file_id, quality):
    
    # Read in ABI AOD data
    aod_data = file_id.variables['AOD'][:,:]
    
    # Select quality of AOD data using the "DQF" variable
    # high quality: DQF = 0, medium quality: DQF = 1, low quality: DQF = 2, not retrieved (NR): DQF = 3
    # NumPy masked_where operation applies "quality_mask" to AOD data array
    dqf = file_id.variables['DQF'][:,:]
    if quality == 'high':
        quality_mask = (dqf != 0)
    elif quality == 'top2':
        quality_mask = (dqf > 1)
    elif quality == 'all':
        quality_mask = (dqf > 2)
    abi_aod = np.ma.masked_where(quality_mask, aod_data)
    
    # Read in GOES ABI fixed grid projection variables and constants
    x_coordinate_1d = file_id.variables['x'][:]
    y_coordinate_1d = file_id.variables['y'][:]
    projection_info = file_id.variables['goes_imager_projection']
    lon_origin = projection_info.longitude_of_projection_origin
    H = projection_info.perspective_point_height+projection_info.semi_major_axis
    r_eq = projection_info.semi_major_axis
    r_pol = projection_info.semi_minor_axis
    
    # Create 2D coordinate matrices from 1D coordinate vectors
    x_coordinate_2d, y_coordinate_2d = np.meshgrid(x_coordinate_1d, y_coordinate_1d)
    
    # Equations to calculate latitude and longitude
    lambda_0 = (lon_origin*np.pi)/180.0  
    a_var = np.power(np.sin(x_coordinate_2d),2.0) + (np.power(np.cos(x_coordinate_2d),2.0)*(np.power(np.cos(y_coordinate_2d),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(y_coordinate_2d),2.0))))
    b_var = -2.0*H*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    c_var = (H**2.0)-(r_eq**2.0)
    r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)
    s_x = r_s*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    s_y = - r_s*np.sin(x_coordinate_2d)
    s_z = r_s*np.cos(x_coordinate_2d)*np.sin(y_coordinate_2d)
    
    abi_lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
    abi_lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)
    
    return abi_aod, abi_lat, abi_lon

In [ ]:
# Block 3: Process ABI AOD data composite (>1 data file) & calculate latitude/longitude from GOES ABI fixed grid projection data
# GOES ABI fixed grid projection is a map projection relative to the GOES satellite
# Units: latitude in °N (°S < 0), longitude in °E (°W < 0)
# See GOES-R Product User Guide (PUG) Volume 5 (L2 products) Section 4.2.8 for details & example of calculations
# "file_list": parameter variable assigned when maps are generated
# "quality": parameter variable from widget menus, set in main function

def abi_aod_composite(file_list, quality):

    # Read in AOD & DQF data, select quality of AOD data using DQF
    # high quality: DQF = 0, medium quality: DQF = 1, low quality: DQF = 2, not retrieved (NR): DQF = 3
    # NumPy masked_where operation applies "quality_mask" to AOD data array
    for i, fname in enumerate(file_list):
        file_id = Dataset(fname)
        aod_data = file_id.variables['AOD'][:,:]
        dqf = file_id.variables['DQF'][:,:]
        if quality == 'high':
            quality_mask = (dqf != 0)
        elif quality == 'top2':
            quality_mask = (dqf > 1)
        elif quality == 'all':
            quality_mask = (dqf > 2)
        aod = np.ma.masked_where(quality_mask, aod_data)

        # Calculate AOD composite
        # NumPy masked zeros operation ("np.ma.zeros") returns a new array of specified shape, filled with zeros
        # "+=1" adds 1 to the blank_composite or composite arrays
        if i == 0:
            blank_composite = np.ma.zeros((aod.shape[0], aod.shape[1]))
            blank_composite.mask = aod.mask
            blank_composite += 1
            # Numpy masked filled operation ("aod.filled(0)") replaces masked AOD data with zeros
            composite = np.ma.zeros((aod.shape[0], aod.shape[1]))
            composite += aod.filled(0)
        else:
            blank_composite.mask = aod.mask
            blank_composite += 1

            composite += aod.filled(0)
        
        # Close AOD data file
        file_id.close()
    
    # Calculate final composite array
    blank_composite.mask = False  # unmask array
    composite = np.ma.masked_equal(composite, 0)  # mask all cells=0
    composite = composite/blank_composite
    
    # Read in GOES ABI fixed grid projection variables and constants
    file_id = Dataset(fname)  # Open last AOD data file
    x_coordinate_1d = file_id.variables['x'][:]
    y_coordinate_1d = file_id.variables['y'][:]
    projection_info = file_id.variables['goes_imager_projection']
    lon_origin = projection_info.longitude_of_projection_origin
    H = projection_info.perspective_point_height+projection_info.semi_major_axis
    r_eq = projection_info.semi_major_axis
    r_pol = projection_info.semi_minor_axis
   
    # Create 2D coordinate matrices from 1D coordinate vectors
    x_coordinate_2d, y_coordinate_2d = np.meshgrid(x_coordinate_1d, y_coordinate_1d)
    
    # Equations to calculate latitude and longitude
    lambda_0 = (lon_origin*np.pi)/180.0  
    a_var = np.power(np.sin(x_coordinate_2d),2.0) + (np.power(np.cos(x_coordinate_2d),2.0)*(np.power(np.cos(y_coordinate_2d),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(y_coordinate_2d),2.0))))
    b_var = -2.0*H*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    c_var = (H**2.0)-(r_eq**2.0)
    r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)
    s_x = r_s*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    s_y = - r_s*np.sin(x_coordinate_2d)
    s_z = r_s*np.cos(x_coordinate_2d)*np.sin(y_coordinate_2d)
    
    abi_lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
    abi_lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)
    
    file_id.close()  # Close AOD data file
    
    return composite, abi_lat, abi_lon

In [ ]:
# Block 4: Define native geostationary map projection from GOES ABI fixed grid projection data
# "file_id": parameter variable assigned when the satellite data file is opened

def projection_information(file_id):
    
    # Read in GOES ABI projection constants
    projection_constants = file_id.variables['goes_imager_projection']
    central_lon = projection_constants.longitude_of_projection_origin
    sat_height = projection_constants.perspective_point_height
    semi_major_axis = projection_constants.semi_major_axis
    semi_minor_axis = projection_constants.semi_minor_axis
    
    # Define geostationary projection using ABI projection constants
    globe = ccrs.Globe(semimajor_axis=semi_major_axis, semiminor_axis=semi_minor_axis)
    geostationary = ccrs.Geostationary(central_longitude=central_lon, satellite_height=sat_height, globe=globe, sweep_axis='x')
    
    return geostationary

In [ ]:
# Block 5: Create AOD color bar, independent of plotted data, auto-positioning
# Color bar location/dimensions set by ".set_position([x0, y0, width, height])" to scale automatically with figure
# Using Matplotlib "rainbow" sequential color map, with "darkred" color for AOD > 1
# "fig", "ax": parameter variables assigned in the plotting function

def aod_colorbar(fig, ax):
    last_axes = plt.gca()
    cbar_ax = fig.add_axes([0, 0, 0, 0])
    plt.draw()
    posn = ax.get_position()
    cbar_ax.set_position([0.35, posn.y0 - 0.06, 0.3, 0.02])
    norm = mpl.colors.Normalize(vmin=0, vmax=1)
    color_map = plt.get_cmap('rainbow').with_extremes(over='darkred')
    cb = mpl.colorbar.ColorbarBase(cbar_ax, cmap=color_map, norm=norm, orientation='horizontal', ticks=[0, 0.25, 0.5, 0.75, 1], 
                                   extend='max')
    cb.set_label(label='Aerosol Optical Depth', size=8, weight='bold')
    cb.ax.set_xticklabels(['0', '0.25', '0.50', '0.75', '1.0'])
    cb.ax.tick_params(labelsize=8)
    plt.sca(last_axes)

In [ ]:
# Block 6: Create array to specify number and range of AOD data contours for plot
# NumPy arange creation routine ("np.arange") returns an array containing evenly spaced values within a given interval
# AOD range for plot = [0,1]; contour interval = 0.05

def data_contours():    
    contours_array = np.arange(0, 1+0.05, 0.05)
    
    return contours_array

In [ ]:
# Block 7: Draw coastlines and borders on map using Natural Earth Feature shapefiles
# "ax": parameter variable assigned in the plotting function 

def map_coastlines_borders(ax):
    
    # "zorder" argument sets drawing order for layers (higher zorder layers plot on top)
    # Set "scale=110m" for lower resolution or "scale=10m" for highest resolution borders & polygons
    ax.coastlines(resolution='50m', zorder=3)
    ax.add_feature(cfeature.NaturalEarthFeature(category='physical', name='ocean', scale='50m'), facecolor='lightgrey')
    ax.add_feature(cfeature.NaturalEarthFeature(category='physical', name='land', scale='50m'), facecolor='grey')
    ax.add_feature(cfeature.NaturalEarthFeature(category='physical', name='lakes', scale='50m'), facecolor='lightgrey', edgecolor='none', zorder=1.5)
    ax.add_feature(cfeature.NaturalEarthFeature(category='physical', name='lakes', scale='50m'), facecolor='none', lw=0.5, edgecolor='black', zorder=2.5)
    ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_0_countries', scale='50m'), facecolor='none', lw=0.75, edgecolor='black', zorder=3)
    # To add state/province borders, uncomment the line below
    ##ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces', scale='50m'), facecolor='none', lw=0.5, edgecolor='black', zorder=3)

In [ ]:
# Block 8: Set map domain and lat/lon tick marks for Plate Carree map projection, using settings entered by user
# "ax": parameter variable assigned in the plotting function
# "lon_ticks", "lat_ticks", "domain": parameter variables assigned when maps are generated

def plate_carree_map_grid(ax, lon_ticks, lat_ticks, domain):
    
    # Format lat/lon tick marks using Matplotlib
    # Important: "crs=ccrs.PlateCarree()" argument required b/c ticks entered as lat/lon values
    # "Plate Carree" map projection in Cartopy displays data entered in geographic coordinates (e.g., lat/lon) correctly
    ax.xaxis.set_major_formatter(LongitudeFormatter())
    ax.yaxis.set_major_formatter(LatitudeFormatter())
    ax.set_xticks(lon_ticks, crs=ccrs.PlateCarree())
    ax.set_yticks(lat_ticks, crs=ccrs.PlateCarree())
    ax.tick_params(length=5, direction='out', labelsize=8, pad=5)
    
    # To add lat/lon gridlines, uncomment the lines below
    ##gl = ax.gridlines(linewidth=0.4, color='silver', zorder=3)
    ##gl.xlocator = ticker.FixedLocator(lon_ticks)
    ##gl.ylocator = ticker.FixedLocator(lat_ticks)
    
    # Set lat/lon boundaries for map domain
    # Important: "crs=ccrs.PlateCarree()" argument required b/c domain boundaries entered as lat/lon values
    # "Plate Carree" map projection in Cartopy displays data entered in geographic coordinates (e.g., lat/lon) correctly
    ax.set_extent(domain, crs=ccrs.PlateCarree())

In [ ]:
# Block 9: Set map domain and lat/lon ticks for native geostationary map projection
# "ax": parameter variable assigned in the plotting function
# "fname": parameter variable assigned when maps are generated

def geostationary_map_grid(ax, fname):
    
    # Get GOES satellite number & ABI scan sector abbreviation from file name
    sat_number = str(fname).split('_')[-4][1:]
    sector = str(fname).split('_')[-5][-4:-3]
    
    # Set lat/lon ticks and map domain
    if sat_number == '16':
        if sector == 'C':
            lon_ticks = [-120, -100, -80, -60]
            lat_ticks = [20, 30, 40, 50]
            ax.set_extent([-115, -60, 15, 53.5], crs=ccrs.PlateCarree())
        elif sector == 'F':
            lon_ticks = [-130, -100, -75, -50, -20]
            lat_ticks = [-60, -40, -20, 0, 20, 40, 60]
            ax.set_global()
    elif sat_number == '17' or sat_number == '18':
        if sector == 'C':
            lon_ticks = [-170, -150, -130, -110]
            lat_ticks = [20, 30, 40, 50]
            ax.set_extent([-162.5, -112.2, 15, 52], crs=ccrs.PlateCarree())
        elif sector == 'F':
            lon_ticks = [170, -160, -137, -110, -80]
            lat_ticks = [-60, -40, -20, 0, 20, 40, 60]
            ax.set_global()
        
    # Format lat/lon grid labels and gridlines using Cartopy
    gl = ax.gridlines(draw_labels=True, linewidth=0.25, color='silver')
    gl.xlines=None  # To draw lon gridlines, comment-out this line 
    gl.ylines=None  # To draw lat gridlines, comment-out this line
    gl.rotate_labels = None
    gl.xlocator = ticker.FixedLocator(lon_ticks)
    gl.ylocator = ticker.FixedLocator(lat_ticks)
    gl.xlabel_style = {'size': 8}
    gl.ylabel_style = {'size': 8}
    if sector == 'F':
        gl.right_labels = None
        gl.x_inline = True
        gl.ypadding = 40  # This ajusts padding of lat labels from edge of globe
    elif sector == 'C':
        gl.bottom_labels = None
        gl.left_labels = None
        if sat_number == '17' or sat_number == '18':
            gl.right_labels = None  # Hide lat labels for G17/G18 domain b/c Cartopy bug duplicates them

In [ ]:
# Block 10: Create title for single file plot from ABI data file name
# "fname": parameter variable assigned when maps are generated
# "quality": parameter variable from widget menus, set in main function

def single_plot_title(fname, quality):
    
    # Set AOD quality level
    if quality == 'high':
        quality_title = '(High Quality AOD)'
    elif quality == 'top2':
        quality_title = '(High & Medium Quality AOD)'
    elif quality == 'all':
        quality_title = '(High, Medium, & Low Quality AOD)'
    
    # Pull Julian date from file name, convert to Gregorian date, and reformat
    julian = datetime.datetime.strptime(str(fname).split('_')[-3][1:8], '%Y%j').date()
    date = julian.strftime('%d %b %Y')

    # Create plot title
    abi_title = 'GOES-'+ str(fname).split('_')[-4][1:] + '/ABI Aerosol Optical Depth  ' + date + ' ' + str(fname).split('_')[-3][8:10] + ':' + str(fname).split('_')[-3][10:12] + ' UTC\n' +  quality_title
    
    return abi_title

In [ ]:
# Block 11: Create title for composite plot from ABI data file names
# Also get composite start/end times to use in saved file name in plotting function
# "file_list": parameter variable assigned when maps are generated
# "quality": parameter variable from widget menus, set in main function

def composite_plot_title(file_list, quality):
    
    # Get first and last AOD file in list of files used to create composite
    first_file = file_list[0]
    last_file = file_list[-1]
    
    # Get start/end times for composite from AOD file names
    start_hour = str(first_file).split('_')[-3][8:10]
    start_min = str(first_file).split('_')[-3][10:12]
    start =  start_hour + ':' + start_min
    start_filename = start_hour + start_min
    end_hour = str(last_file).split('_')[-3][8:10]
    end_min = str(last_file).split('_')[-3][10:12]
    end =  end_hour + ':' + end_min
    end_filename = end_hour + end_min
    
    # Pull Julian date from file name, convert to Gregorian date, and reformat
    julian = datetime.datetime.strptime(str(first_file).split('_')[-3][1:8], '%Y%j').date()
    date = julian.strftime('%d %b %Y')
    
    # Set AOD quality level
    if quality == 'high':
        quality_title = '(High Quality AOD)'
    elif quality == 'top2':
        quality_title = '(High & Medium Quality AOD)'
    elif quality == 'all':
        quality_title = '(High, Medium, & Low Quality AOD)'

    # Create composite plot title
    abi_composite_title = 'GOES-'+ str(first_file).split('_')[-4][1:] + '/ABI Aerosol Optical Depth  ' + date + '  ' + start + '-' + end + ' UTC Composite\n' + quality_title
    
    return abi_composite_title, first_file, start_filename, end_filename

In [ ]:
# Block 12: Plot ABI AOD data from single file on Plate Carree map projection using domain & lat/lon ticks entered by user
# "file_id": parameter variable assigned when the satellite data file is opened
# "fname": parameter variable assigned when maps are generated
# "quality", "save_format", "save_path", "file_res": parameter variables widget menus, set in main function
# "lon_ticks", "lat_ticks", "domain": paramter variables set when maps are generated

def plot_abi_aod_single_pc(fname, file_id, quality, save_format, save_path, file_res, lon_ticks, lat_ticks, domain):
    
    # Set up figure and map projection (Plate Carree)
    # Plate Carree: equidistant cylindrical projection w/equator as the standard parallel; default central_longitude=0
    fig = plt.figure(figsize=(8, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    # Format map appearance
    map_coastlines_borders(ax)
    
    # Set map domain, lat/lon ticks (Plate Carree settings)
    plate_carree_map_grid(ax, lon_ticks, lat_ticks, domain)
    
    # Add figure title
    plt.title(single_plot_title(fname, quality), pad=10, ma='center', size=8, weight='bold')
    
    # Add AOD color bar
    aod_colorbar(fig, ax)
    
    # Process AOD data, calculate latitude and longitude
    aod, lat, lon = abi_aod_data(file_id, quality)

    # Plotting settings for AOD data
    contours_array = data_contours()
    color_map = plt.get_cmap('rainbow').with_extremes(over='darkred')
    
    # Create filled contour plot of AOD data
    # Important: "transform=ccrs.PlateCarree()" argument required b/c AOD data are in geographic (lat/lon) coordinates
    # "try/except" prevents error if "aod" array contains no unmasked data
    try:
        ax.contourf(lon, lat, aod, contours_array, cmap=color_map, extend='both', zorder=2.5, transform=ccrs.PlateCarree())
    except:
        pass
    
    # Show figure
    plt.show()
    
    # Save figure with "save_name" to specified directory using specified data format
    date = datetime.datetime.strptime(str(fname).split('_')[-3][1:8], '%Y%j').date().strftime('%Y%m%d')
    save_name = 'g' + str(fname).split('_')[-4][1:] + '_abi_aod_' + quality + '_' + date + '_' + str(fname).split('_')[-3][8:12] + '_zoom-in' + save_format
    # Set save_path + save_name as pathlib.Path object and convert to string
    full_save = str(save_path / save_name)
    fig.savefig(full_save, bbox_inches='tight', dpi=file_res)
    
    # Erase plot so we can build the next one
    plt.close()

In [ ]:
# Block 13: Plot ABI AOD data from single file on the native geostationary map projection (automatic lat/lon tick marks)
# "file_id": parameter variable assigned when the satellite data file is opened
# "fname": parameter variable assigned when maps are generated
# "quality", "save_format", "save_path", "file_res": parameter variables widget menus, set in main function

def plot_abi_aod_single_geo(fname, file_id, quality, save_format, save_path, file_res):
    
    # Define native ABI geostationary projection
    geostationary = projection_information(file_id)
    
    # Set up figure and map projection (geostationary)
    fig = plt.figure(figsize=(8, 10))
    ax = plt.axes(projection=geostationary)
    
    # Format map appearance
    map_coastlines_borders(ax)
    
    # Set map domain, lat/lon ticks (geostationary settings)
    geostationary_map_grid(ax, fname)
    
    # Add figure title
    plt.title(single_plot_title(fname, quality), pad=10, ma='center', size=8, weight='bold')
    
    # Add AOD color bar
    aod_colorbar(fig, ax)
    
    # Process AOD data, calculate latitude and longitude
    aod, lat, lon = abi_aod_data(file_id, quality)

    # Plotting settings for AOD data
    contours_array = data_contours()
    color_map = plt.get_cmap('rainbow').with_extremes(over='darkred')
    
    # Create filled contour plot of AOD data
    # Important: "transform=ccrs.PlateCarree()" argument required b/c AOD data are in geographic (lat/lon) coordinates
    # "try/except" prevents error if "aod" array contains no unmasked data
    try:
        ax.contourf(lon, lat, aod, contours_array, cmap=color_map, extend='both', zorder=2.5, transform=ccrs.PlateCarree())
    except:
        pass
    
    # Show figure
    plt.show()
    
    # Save figure with "save_name" to specified directory using specified data format
    date = datetime.datetime.strptime(str(fname).split('_')[-3][1:8], '%Y%j').date().strftime('%Y%m%d')
    sector = str(fname).split('_')[-5][-4:-3]
    if sector == 'F':
        sector_save_name = '_fulldisk'
    elif sector == 'C':
        sector_save_name = '_conus'
    save_name = 'g' + str(fname).split('_')[-4][1:] + '_abi_aod_' + quality + '_' + date + '_' + str(fname).split('_')[-3][8:12] + sector_save_name + save_format
    # Set save_path + save_name as pathlib.Path object and convert to string
    full_save = str(save_path / save_name)
    fig.savefig(full_save, bbox_inches='tight', dpi=file_res)
    
    # Erase plot so we can build the next one
    plt.close()

In [ ]:
# Block 14: Plot ABI AOD data composite on Plate Carree map projection using domain & lat/lon ticks entered by user
# "file_list": parameter variable assigned when maps are generated
# "quality", "save_format", "save_path", "file_res": parameter variables from widget menus, set in main function
# "lon_ticks", "lat_ticks", "domain": paramter variables set when maps are generated

def plot_abi_aod_composite_pc(file_list, quality, save_format, save_path, file_res, lon_ticks, lat_ticks, domain):
    
    # Set up figure and map projection (Plate Carree)
    # Plate Carree: equidistant cylindrical projection w/equator as the standard parallel; default central_longitude=0
    fig = plt.figure(figsize=(8, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    # Format map appearance
    map_coastlines_borders(ax)
    
    # Set map domain, lat/lon ticks (Plate Carree settings)
    plate_carree_map_grid(ax, lon_ticks, lat_ticks, domain)
    
    # Add AOD color bar
    aod_colorbar(fig, ax)

    # Plotting settings for AOD data
    contours_array = data_contours()
    color_map = plt.get_cmap('rainbow').with_extremes(over='darkred')
    
    # Process AOD data, calculate latitude and longitude
    aod, lat, lon = abi_aod_composite(file_list, quality)

    # Create filled contour plot of composite AOD data
    ax.contourf(lon, lat, aod, contours_array, cmap=color_map, extend='both', zorder=2.5, transform=ccrs.PlateCarree())

    # Add plot title, format save_name
    abi_composite_title, first_file, start_filename, end_filename = composite_plot_title(file_list, quality)
    date = datetime.datetime.strptime(str(first_file).split('_')[-3][1:8], '%Y%j').date().strftime('%Y%m%d')
    save_name = 'g' + str(first_file).split('_')[-4][1:] + '_abi_aod_composite_' + quality + '_' + date + '_' + start_filename + '-' + end_filename + '_zoom-in' + save_format
    plt.title(abi_composite_title, pad=10, ma='center', size=8, weight='bold')

    # Show figure
    plt.show()

    # Save figure with "save_name" to specified directory using specified data format
    # Set save_path + save_name as pathlib.Path object and convert to string
    full_save = str(save_path / save_name)
    fig.savefig(full_save, bbox_inches='tight', dpi=file_res)
    
    # Erase plot so we can build the next one
    plt.close()

In [ ]:
# Block 15: Plot ABI AOD data composite on the native geostationary map projection (automatic lat/lon tick marks)
# "file_list": parameter variable assigned when maps are generated
# "quality", "save_format", "save_path", "file_res": parameter variables from widget menus, set in main function

def plot_abi_aod_composite_geo(file_list, quality, save_format, save_path, file_res):
    
    # Define native ABI geostationary projection
    fname = file_list[0]
    file_id = Dataset(fname)
    geostationary = projection_information(file_id)
    
    # Set up figure and map projection (geostationary)
    fig = plt.figure(figsize=(8, 10))
    ax = plt.axes(projection=geostationary)
    
    # Format map appearance
    map_coastlines_borders(ax)
    
    # Set map domain, lat/lon ticks (geostationary settings)
    geostationary_map_grid(ax, fname)
    
    # Add AOD color bar
    aod_colorbar(fig, ax)

    # Plotting settings for AOD data
    contours_array = data_contours()
    color_map = plt.get_cmap('rainbow').with_extremes(over='darkred')
    
    # Process AOD data, calculate latitude and longitude
    aod, lat, lon = abi_aod_composite(file_list, quality)

    # Create filled contour plot of composite AOD data
    ax.contourf(lon, lat, aod, contours_array, cmap=color_map, extend='both', zorder=2.5, transform=ccrs.PlateCarree())

    # Add plot title, format save_name
    abi_composite_title, first_file, start_filename, end_filename = composite_plot_title(file_list, quality)
    date = datetime.datetime.strptime(str(first_file).split('_')[-3][1:8], '%Y%j').date().strftime('%Y%m%d')
    sector = str(first_file).split('_')[-5][-4:-3]
    if sector == 'F':
        sector_save_name = '_fulldisk'
    elif sector == 'C':
        sector_save_name = '_conus'
    save_name = 'g' + str(first_file).split('_')[-4][1:] + '_abi_aod_composite_' + quality + '_' + date + '_' + start_filename + '-' + end_filename + sector_save_name + save_format
    plt.title(abi_composite_title, pad=10, ma='center', size=8, weight='bold')

    # Show figure
    plt.show()

    # Save figure with "save_name" to specified directory using specified data format
    # Set save_path + save_name as pathlib.Path object and convert to string
    full_save = str(save_path / save_name)
    fig.savefig(full_save, bbox_inches='tight', dpi=file_res)
    
    # Erase plot so we can build the next one
    plt.close()

In [ ]:
# Block 16: User interface to process/visualize AOD data
# "domain_option", "file_path", "processing", "quality", "save_format", "save_path", "file_res": parameter variables from widget menus, set in main function

def process_visualize_data(domain_option, file_path, processing, quality, save_format, save_path, file_res):
        
        # Print directory where map files will be saved (so user can check before proceeding)
        print('\nMap files will be saved to: ' + str(save_path))
        
        if domain_option == 1:
            print('\nFull extent of AOD data will be plotted on the native ABI geostationary map projection.')
        elif domain_option == 2:  
            # Create list of floats for map domain boundaries
            domain = [west_lon.value, east_lon.value, south_lat.value, north_lat.value]
            # Create lists of floats for latitude and longitude tick marks
            lon_ticks = supporting_functions.generate_ticks(lon_start.value, lon_stop.value, lon_increment.value)
            lat_ticks = supporting_functions.generate_ticks(lat_start.value, lat_stop.value, lat_increment.value)
            # Print manual map domain, lat/lon ticks (so user can check before proceeding)
            print('\nA subset of AOD data will be plotted on the Plate Carree map projection.')
            print('\nSpecified map domain corners are (western lon, eastern lon, southern lat, northern lat):', domain)
            print('\nSpecified map longitude ticks are (west-to-east):', lon_ticks)
            print('\nSpecified map latitude ticks are (south-to-north):', lat_ticks)
        
        # Ask if user wants to process/visualze data
        # If yes, proceed to make maps
        make_maps = input('\nWould you like to proceed with making the ABI AOD map(s)?\nType "yes" or "no" and hit "Enter"\n')
        if make_maps in ['yes', 'YES', 'Yes', 'y', 'Y']:
            print('Processing and visualizing AOD data...')
            
            # Collect all of the AOD netCDF4 files in the specified directory
            # Set path for files (using wildcard) as pathlib.Path object
            file_list = sorted(file_path.glob('*AOD*.nc')) 
            # If no files found, notify user
            if len(file_list) == 0:
                print('\nNo AOD netCDF files found in specified directory. Try again.')
            else:
                if processing == 'single':
                    # Loop through AOD data files, making/saving a map figure for each file
                    for fname in file_list:    
                        # Open the data file
                        file_id = Dataset(fname)
                        # Process/plot data
                        if domain_option == 1:
                            plot_abi_aod_single_geo(fname, file_id, quality, save_format, save_path, file_res)
                        elif domain_option == 2:
                            plot_abi_aod_single_pc(fname, file_id, quality, save_format, save_path, file_res, lon_ticks, lat_ticks, domain)
                        # Close the data file
                        file_id.close()   
                    print('Done!')
                elif processing == 'composite':
                    if domain_option == 1:
                        plot_abi_aod_composite_geo(file_list, quality, save_format, save_path, file_res)
                    elif domain_option == 2:
                        plot_abi_aod_composite_pc(file_list, quality, save_format, save_path, file_res, lon_ticks, lat_ticks, domain)
                    print('Done!')
        else:
            print('\nUser has opted not to make AOD maps.')

In [ ]:
# Block 17: Enter satellite data processing & visualization settings using interactive Jupyter Notebook widgets

# Run this block *once* to generate menus
# When main function is run, it reads "(widget-menu-variable).value" of each menu selection
# Do NOT re-run block if you change menu selections! Re-running block resets menus to defaults!

# Create radiobutton menu to select directory where ABI data files are located
file_path_caption = widgets.Label(value='SELECT DIRECTORY where ABI data files are located', layout=widgets.Layout(height='20px'))
file_path_radiobutton = widgets.RadioButtons(options=[('Current Working Directory', 1), ('Specify a Directory: (e.g., D://Data/)', 2)], disabled=False, layout=widgets.Layout(height='40px'))
file_path_directory = widgets.Text(disabled=True, layout=widgets.Layout(width='500px', height='30px'))

# Function to enable text entry only if file_path_radiobutton to "specify a directory" is selected
def handle_file_path_directory_change(change):
    file_path_directory.disabled=False if change.new == 2 else True

# Monitor values of file_path_radiobutton
file_path_radiobutton.observe(handle_file_path_directory_change, names='value')

# Create radiobutton menu to select directory where map files will be saved
save_path_caption = widgets.Label(value='SELECT DIRECTORY where map image files will be saved', layout=widgets.Layout(height='20px'))
save_path_radiobutton = widgets.RadioButtons(options=[('Current Working Directory', 1), ('Specify a Directory: (e.g., D://Data/)', 2)], disabled=False, layout=widgets.Layout(height='40px'))
save_path_directory = widgets.Text(disabled=True, layout=widgets.Layout(width='500px', height='30px'))

# Function to enable text entry only if save_path_radiobutton to "specify a directory" is selected
def handle_save_path_directory_change(change):
    save_path_directory.disabled=False if change.new == 2 else True

# Monitor values of radiobuttons
file_path_radiobutton.observe(handle_file_path_directory_change, names='value')
save_path_radiobutton.observe(handle_save_path_directory_change, names='value')

# Display directory widgets
print('If you change entries or menu selections (e.g., to create additional maps), do NOT re-run this block!\nRe-running will reset all menus to their defaults!')
display(file_path_caption, file_path_radiobutton, file_path_directory)
display(save_path_caption, save_path_radiobutton, save_path_directory)

# Formatting settings for drop-down menus
style = {'description_width':'220px'}
layout = widgets.Layout(width='420px')

# Caption for drop-down menus
drop_menu_caption = widgets.Label(value='SELECT settings for AOD data visualization', layout=widgets.Layout(height='20px'))

# Create drop-down menus
processing = widgets.Dropdown(options=[('Individual AOD files', 'single'), ('AOD multi-file composite', 'composite')], description='AOD map type:', style=style, layout=layout)
quality = widgets.Dropdown(options=[('High', 'high'), ('High & Medium ("Top 2")', 'top2'), ('High, Medium & Low ("All")', 'all')], description='AOD data quality:', style=style, layout=layout)
save_format = widgets.Dropdown(options=[('.png'), ('.jpg'), ('.pdf')], description='AOD map image file format:', style=style, layout=layout)
file_res = widgets.Dropdown(options=[(150), (300), (450), (600), (900)], description='AOD map image file resolution (DPI):', style=style, layout=layout)

# Additonal labels to explain menu values
map_file_res_label = widgets.Label(value='(low resolution = 150 DPI, very high resolution = 900 DPI)', layout=widgets.Layout(width='400px'))

# Format menus and labels to display side-by-side
file_res_box = widgets.HBox([file_res, map_file_res_label])

# Display drop-down menus and caption
display(drop_menu_caption, processing, quality, save_format, file_res_box)

# Create radiobutton menu to select domain for maps
domain_option_caption = widgets.Label(value='CHOOSE METHOD TO SET DOMAIN OF MAPS', layout=widgets.Layout(height='20px'))
domain_option_radiobutton = widgets.RadioButtons(options=[('Plot entire Full Disk or CONUS sector AOD domain (map corners & lat/lon ticks set AUTOMATICALLY)', 1), 
                                                          ('Specify map domain MANUALLY (you need to ENTER SETTINGS BELOW for lat/lon map corners & tick marks!)', 2)], 
                                                 disabled=False, layout=widgets.Layout(width='800px',height='40px'))

# Display radiobutton widgets
display(domain_option_caption, domain_option_radiobutton)

# Caption for map domain boundaries
domain_caption = widgets.Label(value='MANUALLY SET DOMAIN OF MAP (enter lat/lon map corners using up/down arrows or type in value)', layout=widgets.Layout(height='30px'))

# Create numerical (float) text entry widgets for map boundary corners
west_lon = widgets.BoundedFloatText(description='western-most longitude:', value=0, min=-180, max=180, disabled=False, layout=widgets.Layout(width='250px', height='30px'), style={'description_width':'150px'})
east_lon = widgets.BoundedFloatText(description='eastern-most longitude:', value=0, min=-180, max=180, disabled=False, layout=widgets.Layout(width='250px', height='30px'), style={'description_width':'150px'})
lon_label = widgets.Label(value='(use negative values to indicate °W, e.g., 100 °W = -100)', layout=widgets.Layout(width='400px'))
lon_box = widgets.HBox([west_lon, east_lon, lon_label])
north_lat = widgets.BoundedFloatText(description='northern-most latitude:', value=0, min=-90, max=90, disabled=False, layout=widgets.Layout(width='400px', height='30px'), style={'description_width':'300px'})
south_lat = widgets.BoundedFloatText(description='southern-most latitude:', value=0, min=-90, max=90, disabled=False, layout=widgets.Layout(width='400px', height='30px'), style={'description_width':'300px'})
lat_label = widgets.Label(value='(use negative values to indicate °S, e.g., 30 °S = -30)', layout=widgets.Layout(width='400px'))
north_lat_box = widgets.HBox([north_lat, lat_label])
south_lat_box = widgets.HBox([south_lat, lat_label])

# Display map domain boundaries widgets
display(domain_caption, north_lat_box, lon_box, south_lat_box)

# Caption for map lat/lon ticks
ticks_caption = widgets.Label(value='MANUALLY SET LAT/LON TICK MARKS FOR MAP (use up/down arrows or type in value) (ignore this section if you do NOT want ticks on map)', layout=widgets.Layout(height='20px'))

# Create numerical (float) text entry widgets for lon ticks
lon_ticks_label = widgets.Label(value='Enter range of longitude ticks: ', layout=widgets.Layout(width='175px'))
lon_start = widgets.BoundedFloatText(description='western-most longitude tick', value=0, min=-180, max=180, disabled=False, layout=widgets.Layout(width='250px', height='30px'), style={'description_width':'175px'})
lon_stop = widgets.BoundedFloatText(description='eastern-most longitude tick', value=0, min=-180, max=180, disabled=False, layout=widgets.Layout(width='250px', height='30px'), style={'description_width':'175px'})
lon_increment = widgets.BoundedFloatText(description='increment of °longitude between ticks (0.1-50°)', value=10, min=0.1, max=50, disabled=False, layout=widgets.Layout(width='575px', height='30px'), style={'description_width':'500px'})
lon_ticks_box = widgets.HBox([lon_ticks_label, lon_start, lon_stop])

# Create numerical (float) text entry widgets for lat ticks
lat_ticks_label = widgets.Label(value='Enter range of latitude ticks: ', layout=widgets.Layout(width='175px'))
lat_start = widgets.BoundedFloatText(description='southern-most latitude tick', value=0, min=-90, max=90, disabled=False, layout=widgets.Layout(width='430px', height='30px'), style={'description_width':'355px'})
lat_stop = widgets.BoundedFloatText(description='northern-most latitude tick', value=0, min=-90, max=90, disabled=False, layout=widgets.Layout(width='250px', height='30px'), style={'description_width':'175px'})
lat_increment = widgets.BoundedFloatText(description='increment of °latitude between ticks (0.1-60°)', value=10, min=0.1, max=50, disabled=False, layout=widgets.Layout(width='575px', height='30px'), style={'description_width':'500px'})
lat_ticks_box = widgets.HBox([lat_ticks_label, lat_stop])

# Display map lat/lon ticks widgets
display(ticks_caption, lon_ticks_box, lon_increment, lat_ticks_box, lat_start, lat_increment)

In [ ]:
# Block 18: Main Function (import user specifications and process/visualize ABI AOD data)
# Selections from widget menus imported using "(widget-menu-variable).value"

# Main function
if __name__ == "__main__":

    # Set directory where ABI data files are located (as pathlib.Path object)
    if file_path_radiobutton.value == 1:
        file_path = Path.cwd()  # Set current working directory as pathlib.Path object
    else:
        file_path_error_message = supporting_functions.check_directory(file_path_directory.value)  # Check for errors
        if file_path_error_message == 0:
            file_path = Path(ffile_path_directory.value)  # Set user-entered directory as pathlib.Path object
        else:
            file_path = 'error'

    # Set directory to save AOD map files (as pathlib.Path object)
    if save_path_radiobutton.value == 1:  # 1=cwd, 2=user-specified directory
        save_path = Path.cwd()  # Set current working directory as pathlib.Path object
    else:
        save_path_error_message = supporting_functions.check_directory(save_path_directory.value)  # Check for errors
        if save_path_error_message == 0:
            save_path = Path(save_path_directory.value)  # Set user-entered directory as pathlib.Path object
        else:
            save_path = 'error'

    # Notify user if errors in entered directory name(s); if none, proceed to AOD data processing/visualizing
    if file_path == 'error':
        if file_path_error_message == 1:
            print('The entered directory where ABI files are located does not exist. Try again.')
        elif file_path_error_message == 2:
            print('You entered a directory where ABI files are located but the field is blank. Try again.')
        elif file_path_error_message == 3:
            print('There is a syntax error in the name of the directory where ABI files are located. Try again.')
    elif save_path == 'error':
        if save_path_error_message == 1:
            print('The directory entered to save AOD map files does not exist. Try again.')
        elif save_path_error_message == 2:
            print('You entered a directory to save AOD map files but the field is blank. Try again.')
        elif save_path_error_message == 3:
            print('There is a syntax error in the the directory name to save AOD map files. Try again.')
    else:
        # Make map(s) of ABI AOD data
        process_visualize_data(domain_option_radiobutton.value, file_path, processing.value, quality.value, save_format.value, 
                               save_path, file_res.value)